<a href="https://colab.research.google.com/github/monmon831/Computer-Vision/blob/main/ProjekCV(K_Means).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gradio as gr
from PIL import Image, ImageDraw, ImageFont
import numpy as np
from sklearn.cluster import KMeans
import tempfile
import os
from typing import Tuple, List, Optional

In [ ]:
# Global state untuk menyimpan data aplikasi
global_state = {
    "image": None,
    "centers": [],
    "bar_width": 150,
    "current_image": None
}

In [ ]:
def rgb_to_hex(rgb: Tuple[int, int, int]) -> str:
    """
    Konversi nilai RGB ke format HEX
    Args:
        rgb: Tuple berisi nilai (R, G, B)
    Returns:
        String format HEX (contoh: #FF5733)
    """
    return f"#{rgb[0]:02x}{rgb[1]:02x}{rgb[2]:02x}".upper()

In [ ]:
def get_dominant_colors(image: Image.Image, n_colors: int = 5) -> np.ndarray:
    """
    Mencari warna dominan dalam gambar menggunakan K-Means clustering
    Args:
        image: PIL Image object
        n_colors: Jumlah warna dominan yang ingin diekstrak
    Returns:
        Array numpy berisi center warna dominan
    """
    try:
        # Konversi ke RGB dan resize untuk efisiensi
        img = image.convert("RGB")
        # Resize gambar untuk mempercepat proses clustering
        img.thumbnail((300, 300), Image.Resampling.LANCZOS)

        img_np = np.array(img)
        h, w, _ = img_np.shape
        img_flat = img_np.reshape((h * w, 3))

        # Gunakan K-Means untuk clustering warna
        kmeans = KMeans(n_clusters=n_colors, random_state=42, n_init=10)
        kmeans.fit(img_flat)
        centers = kmeans.cluster_centers_.astype(int)

        # Urutkan berdasarkan frekuensi kemunculan
        labels = kmeans.labels_
        unique_labels, counts = np.unique(labels, return_counts=True)
        sorted_indices = np.argsort(counts)[::-1]
        centers = centers[sorted_indices]

        return centers
    except Exception as e:
        print(f"Error dalam get_dominant_colors: {e}")
        return np.array([[255, 0, 0], [0, 255, 0], [0, 0, 255], [255, 255, 0], [255, 0, 255]])

In [ ]:
def generate_palette_image(centers: np.ndarray) -> Image.Image:
    """
    Membuat gambar palet warna dengan informasi RGB dan HEX
    Args:
        centers: Array center warna dari K-Means
    Returns:
        PIL Image berisi palet warna
    """
    bar_width = global_state["bar_width"]
    bar_height = 120
    label_height = 80
    total_width = bar_width * len(centers)
    total_height = bar_height + label_height

    # Buat canvas putih
    palette_img = Image.new("RGB", (total_width, total_height), (255, 255, 255))
    draw = ImageDraw.Draw(palette_img)

    # Load font
    try:
        font_large = ImageFont.truetype("arial.ttf", 14)
        font_small = ImageFont.truetype("arial.ttf", 12)
    except:
        font_large = ImageFont.load_default()
        font_small = ImageFont.load_default()

    # Gambar setiap warna dalam palet
    for i, center in enumerate(centers):
        color = tuple(map(int, center))
        hex_color = rgb_to_hex(color)

        # Gambar bar warna
        draw.rectangle(
            [i * bar_width, 0, (i + 1) * bar_width, bar_height],
            fill=color,
        )

        # Area putih untuk label
        text_bg_y = bar_height
        draw.rectangle(
            [i * bar_width, text_bg_y, (i + 1) * bar_width, text_bg_y + label_height],
            fill=(255, 255, 255)
        )

        # Label RGB
        rgb_text = f"RGB: {color}"
        draw.text((i * bar_width + 5, text_bg_y + 5), rgb_text, fill="black", font=font_small)

        # Label HEX
        draw.text((i * bar_width + 5, text_bg_y + 25), hex_color, fill="black", font=font_large)

        # Label urutan
        draw.text((i * bar_width + 5, text_bg_y + 45), f"Rank #{i+1}", fill="gray", font=font_small)

        # Border untuk memisahkan warna
        draw.rectangle(
            [i * bar_width, 0, (i + 1) * bar_width, total_height],
            outline="lightgray",
            width=2
        )

    return palette_img

In [ ]:
def find_closest_pixels(image: Image.Image, target_rgb: Tuple[int, int, int], num_points: int = 5) -> List[Tuple[int, int]]:
    """
    Mencari beberapa piksel terdekat dengan warna target
    Args:
        image: PIL Image object
        target_rgb: Warna target (R, G, B)
        num_points: Jumlah titik terdekat yang dicari
    Returns:
        List koordinat (x, y) dari piksel terdekat
    """
    img = image.convert("RGB")
    img_np = np.array(img)
    h, w, _ = img_np.shape

    # Hitung jarak euclidean dari setiap piksel ke warna target
    diff = img_np - np.array(target_rgb)
    dist = np.linalg.norm(diff, axis=2)

    # Ambil beberapa posisi dengan jarak terkecil
    flat_indices = np.argpartition(dist.flatten(), num_points)[:num_points]
    positions = []

    for idx in flat_indices:
        y, x = np.unravel_index(idx, dist.shape)
        positions.append((x, y))

    return positions

In [ ]:
def process_image(image: Optional[Image.Image], n_colors: int = 5) -> Tuple[Optional[Image.Image], Optional[Image.Image]]:
    """
    Memproses gambar yang diupload untuk mengekstrak warna dominan
    Args:
        image: PIL Image object dari upload
        n_colors: Jumlah warna dominan yang ingin diekstrak
    Returns:
        Tuple berisi gambar asli dan palet warna
    """
    if image is None:
        return None, None

    try:
        # Ekstrak warna dominan dengan jumlah yang bisa diatur
        centers = get_dominant_colors(image, n_colors=n_colors)
        palette = generate_palette_image(centers)

        # Simpan ke global state
        global_state["image"] = image.copy()
        global_state["centers"] = centers
        global_state["current_image"] = image.copy()

        return image, palette
    except Exception as e:
        print(f"Error dalam process_image: {e}")
        return image, None

In [ ]:
def on_palette_click(evt: gr.SelectData) -> Optional[Image.Image]:
    """
    Handler ketika user mengklik palet warna
    Args:
        evt: Event data dari Gradio SelectData
    Returns:
        Gambar dengan marking pada area warna yang dipilih
    """
    try:
        if global_state["image"] is None or len(global_state["centers"]) == 0:
            return global_state["current_image"]

        # Tentukan warna mana yang diklik
        x, y = evt.index if evt.index else (0, 0)
        bar_width = global_state["bar_width"]
        color_index = min(int(x // bar_width), len(global_state["centers"]) - 1)

        selected_color = tuple(map(int, global_state["centers"][color_index]))

        # Buat copy gambar asli
        marked_img = global_state["image"].copy()
        draw = ImageDraw.Draw(marked_img)

        # Cari beberapa titik terdekat dengan warna yang dipilih
        closest_positions = find_closest_pixels(marked_img, selected_color, num_points=3)

        # Gambar marking pada setiap posisi
        for i, (pos_x, pos_y) in enumerate(closest_positions):
            radius = 25 - (i * 5)  # Radius berbeda untuk setiap titik
            # Lingkaran luar (putih)
            draw.ellipse(
                [pos_x - radius - 2, pos_y - radius - 2, pos_x + radius + 2, pos_y + radius + 2],
                fill="white",
                outline="black",
                width=2
            )
            # Lingkaran dalam (warna target)
            draw.ellipse(
                [pos_x - radius, pos_y - radius, pos_x + radius, pos_y + radius],
                fill=selected_color,
                outline="white",
                width=2
            )

        # Tambahkan informasi warna yang dipilih
        hex_color = rgb_to_hex(selected_color)
        info_text = f"Selected Color: RGB{selected_color} | {hex_color}"

        # Background untuk text info
        try:
            font = ImageFont.truetype("arial.ttf", 16)
        except:
            font = ImageFont.load_default()

        # Hitung ukuran text
        bbox = draw.textbbox((0, 0), info_text, font=font)
        text_width = bbox[2] - bbox[0]
        text_height = bbox[3] - bbox[1]

        # Gambar background text
        padding = 10
        draw.rectangle(
            [10, 10, 10 + text_width + padding * 2, 10 + text_height + padding * 2],
            fill="white",
            outline="black",
            width=2
        )

        # Gambar text
        draw.text((10 + padding, 10 + padding), info_text, fill="black", font=font)

        global_state["current_image"] = marked_img
        return marked_img

    except Exception as e:
        print(f"Error dalam on_palette_click: {e}")
        return global_state["current_image"] if global_state["current_image"] is not None else None

In [ ]:
def save_palette(image: Optional[Image.Image]) -> Optional[str]:
    """
    Menyimpan palet warna ke file temporary
    Args:
        image: PIL Image palet warna
    Returns:
        Path file temporary atau None jika gagal
    """
    if image is None:
        return None

    try:
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".png")
        image.save(temp_file.name, "PNG")
        temp_file.close()
        return temp_file.name
    except Exception as e:
        print(f"Error dalam save_palette: {e}")
        return None

In [ ]:
def reset_app():
    """Reset aplikasi ke state awal"""
    global_state["image"] = None
    global_state["centers"] = []
    global_state["current_image"] = None
    return None, None, None, 5  # Reset slider ke nilai default

In [ ]:
# Gradio Interface
def create_interface():
    """Membuat interface Gradio"""
    with gr.Blocks(
        title="🎨 Deteksi Warna Dominan",
        theme=gr.themes.Soft(),
        css="""
        .gradio-container {
            max-width: 1200px !important;
        }
        """
    ) as demo:

        gr.Markdown("""
        # 🎨 Aplikasi Deteksi Warna Dominan

        **Fitur Utama:**
        - 🔍 Ekstraksi warna dominan dari gambar (2-10 warna)
        - 🎯 Deteksi posisi warna serupa di gambar asli
        - 📊 Informasi lengkap RGB dan HEX
        - 💾 Download palet warna
        - ⚙️ Pengaturan jumlah warna yang fleksibel
        """)

        with gr.Row():
            with gr.Column(scale=1):
                input_image = gr.Image(
                    type="pil",
                    label="📷 Upload Gambar",
                    height=400
                )

                # Slider untuk mengatur jumlah warna dominan
                color_count_slider = gr.Slider(
                    minimum=2,
                    maximum=10,
                    value=5,
                    step=1,
                    label="🎨 Jumlah Warna Dominan",
                    info="Pilih berapa banyak warna dominan yang ingin dideteksi (2-10)",
                    interactive=True
                )

                with gr.Row():
                    clear_btn = gr.Button("🗑️ Clear", variant="secondary", size="sm")
                    process_btn = gr.Button("🔄 Proses Ulang", variant="primary", size="sm")

            with gr.Column(scale=1):
                output_image = gr.Image(
                    label="🖼️ Gambar dengan Marking Warna",
                    height=400
                )

        with gr.Row():
            palette_image = gr.Image(
                label="🎨 Palet Warna Dominan (Klik untuk melihat posisi di gambar)",
                height=220,
                interactive=True
            )

        with gr.Row():
            download_palette = gr.File(
                label="💾 Download Palet Warna",
                visible=False
            )

        gr.Markdown("""
        ### 📋 Cara Penggunaan:

        1. **Upload Gambar** 📸
           - Klik area upload dan pilih gambar (JPG, PNG, dll.)
           - Aplikasi akan otomatis menganalisis gambar

        2. **Atur Jumlah Warna** 🎨
           - Gunakan slider untuk mengatur jumlah warna dominan (2-10)
           - Klik "Proses Ulang" setelah mengubah jumlah warna
           - Semakin banyak warna, semakin detail analisisnya

        3. **Analisis Warna** 🔍
           - Palet warna akan muncul dengan informasi RGB dan HEX
           - Warna diurutkan berdasarkan dominansi (Rank #1 = paling dominan)

        4. **Eksplorasi Posisi Warna** 🎯
           - Klik pada salah satu warna di palet
           - Sistem akan menandai 3 area terdekat dengan warna tersebut di gambar asli
           - Informasi warna yang dipilih akan muncul di bagian atas gambar

        5. **Fitur Tambahan** ⚡
           - Gunakan tombol "Clear" untuk reset aplikasi
           - Palet warna bisa didownload untuk referensi
           - Tombol "Proses Ulang" untuk update hasil tanpa upload ulang

        ### 🛠️ Teknologi yang Digunakan:
        - **K-Means Clustering**: Untuk mengelompokkan warna serupa
        - **Euclidean Distance**: Untuk mencari piksel dengan warna terdekat
        - **PIL (Python Imaging Library)**: Untuk manipulasi gambar
        - **Gradio**: Untuk interface web yang interaktif

        ### 💡 Tips Penggunaan:
        - **2-3 warna**: Cocok untuk gambar dengan warna sederhana
        - **4-6 warna**: Ideal untuk kebanyakan foto
        - **7-10 warna**: Untuk gambar dengan banyak detail warna
        """)

        # Event handlers
        input_image.change(
            fn=process_image,
            inputs=[input_image, color_count_slider],
            outputs=[output_image, palette_image]
        ).then(
            fn=save_palette,
            inputs=[palette_image],
            outputs=[download_palette]
        ).then(
            fn=lambda x: gr.File(visible=bool(x)),
            inputs=[download_palette],
            outputs=[download_palette]
        )

        # Handler untuk proses ulang dengan slider
        process_btn.click(
            fn=process_image,
            inputs=[input_image, color_count_slider],
            outputs=[output_image, palette_image]
        ).then(
            fn=save_palette,
            inputs=[palette_image],
            outputs=[download_palette]
        ).then(
            fn=lambda x: gr.File(visible=bool(x)),
            inputs=[download_palette],
            outputs=[download_palette]
        )

        # Handler untuk mengubah slider (otomatis proses ulang jika ada gambar)
        color_count_slider.change(
            fn=lambda img, n_colors: process_image(img, n_colors) if img is not None else (None, None),
            inputs=[input_image, color_count_slider],
            outputs=[output_image, palette_image]
        ).then(
            fn=save_palette,
            inputs=[palette_image],
            outputs=[download_palette]
        ).then(
            fn=lambda x: gr.File(visible=bool(x)),
            inputs=[download_palette],
            outputs=[download_palette]
        )

        palette_image.select(
            fn=on_palette_click,
            outputs=[output_image]
        )

        clear_btn.click(
            fn=reset_app,
            outputs=[input_image, output_image, palette_image, color_count_slider]
        )

    return demo

In [ ]:
# Jalankan aplikasi
if __name__ == "__main__":
    demo = create_interface()
    demo.launch(
        share=True,
        server_name="0.0.0.0",
        show_error=True
    )

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://762fa074e7b2ea41e9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
